# 8.1 Sparse Matrix의 개념과 Python에서의 사용

In [1]:
import numpy as np
import pandas as pd
#sparse matrix를 사용하기 위한 scipy 라이브러리 
from scipy.sparse import csr_matrix 

#간단한 테스트를 위한 임시 데이터
ratings = {'user_id' : [1, 2, 4],
           'movie_id':[2, 3, 7],
           'rating' : [4, 3, 1]}
ratings = pd.DataFrame(ratings)

#Pandas pivot을 이용해서 full matrix 변환
#일반적인 DataFrame의 pivot 기능을 사용해서 full matrix 변환
rating_matrix = ratings.pivot(index = 'user_id',
                              columns = 'movie_id',
                              values = 'rating').fillna(0)
      
full_matrix1 = np.array(rating_matrix)
print(full_matrix1)

[[4. 0. 0.]
 [0. 3. 0.]
 [0. 0. 1.]]


In [2]:
rating_matrix

movie_id,2,3,7
user_id,,,
1,4.0,0.0,0.0
2,0.0,3.0,0.0
4,0.0,0.0,1.0


In [3]:
#Sparse matrix를 이용해서 full matrix 변환
#원소의 값(평점) 지정 
data = np.array(ratings['rating'])
#row 인덱스 지정 
row_indices = np.array(ratings['user_id'])
#column 인덱스 지정
col_indices = np.array(ratings['movie_id'])
#원래 데이터를 아까 설명했던 csr_matrix로 변환 
rating_matrix = csr_matrix((data,(row_indices, col_indices)),dtype=int)
print(rating_matrix)

  (1, 2)	4
  (2, 3)	3
  (4, 7)	1


In [4]:
rating_matrix[1, 2]

4

In [5]:
full_matrix2 = rating_matrix.toarray()
print(full_matrix2)

[[0 0 0 0 0 0 0 0]
 [0 0 4 0 0 0 0 0]
 [0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]]


# 8.2 Sparse Matrix를 추천 알고리즘에 적용하기

In [1]:
#데이터가 너무 길어서 오류가 뜸 

from sklearn.model_selection import train_test_split
import random
import numpy as np
import pandas as pd
import os 

base_src = './Data'
ratings_20m_src = os.path.join(base_src, 'ratings-20m.csv')
r_cols = ["user_id","movie_id",'rating','timestamp']
#20M data 읽어오기 
ratings = pd.read_csv(ratings_20m_src,
                    sep = ',',
                    names = r_cols,
                    encoding='latin-1')

R_temp = ratings.pivot(index='user_id', columns = 'movie_id', values = 'rating').fillna(0)

ValueError: Unstacked DataFrame is too big, causing int32 overflow